In [1]:
!pip install owslib geopandas requests matplotlib folium --quiet

In [2]:
from owslib.wms import WebMapService
from owslib.wfs import WebFeatureService
import geopandas as gpd
import requests
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import folium
from IPython.display import Image, display
import geopandas as gpd
import requests


/home/vishnu/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### Fetching geoserver URL for vector layer 

In [3]:
GEOSERVER_URL = "https://geoserver.core-stack.org:8443/geoserver" 

In [4]:
def get_vector_url(workspace, layer_name):
    
    geojson_url = (
        f"{GEOSERVER_URL}/{workspace}/ows?"
        f"service=WFS&version=1.0.0&request=GetFeature&"
        f"typeName={workspace}:{layer_name}&outputFormat=application/json"
    )
    print("GeoJSON URL:", geojson_url)
    return geojson_url


In [5]:

workspace = "cropping_drought"
layer_name = "jaunpur_badlapur_drought"

geojson_url = get_vector_url(workspace, layer_name)

GeoJSON URL: https://geoserver.core-stack.org:8443/geoserver/cropping_drought/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=cropping_drought:jaunpur_badlapur_drought&outputFormat=application/json


### fetching geoserver link for raster  

In [6]:
import geopandas as gpd

GEOSERVER_URL = "https://geoserver.core-stack.org:8443/geoserver"


In [14]:
def get_raster_url(workspace, layer_name, output_format="geotiff"):
    wcs_url = (
        f"{GEOSERVER_URL}/{workspace}/wcs?"
        f"service=WCS&version=2.0.1&request=GetCoverage&"
        f"CoverageId={workspace}:{layer_name}&"
        f"format={output_format}"
    )
    print("Raster URL:",wcs_url)
    return wcs_url

    

In [20]:
## LULC 
# workspace = "LULC_level_1"
# layer_name = "LULC_23_24_badlapur_level_1"

##terrain
# workspace = "terrain"
# layer_name = "jaunpur_badlapur_terrain_raster"

##clart
# workspace = "clart"
# layer_name = "jaunpur_badlapur_clart"

##tree health ccd
# workspace = "ccd"
# layer_name = "tree_health_ccd_raster_jaunpur_badlapur_2022"


workspace = "restoration"
layer_name = "restoration_jaunpur_badlapur_raster"

raster_url = get_raster_url(workspace, layer_name)


Raster URL: https://geoserver.core-stack.org:8443/geoserver/restoration/wcs?service=WCS&version=2.0.1&request=GetCoverage&CoverageId=restoration:restoration_jaunpur_badlapur_raster&format=geotiff


## To get the geoserver URL if we pass state,district & block information  

In [7]:
import pandas as pd


In [23]:
df = pd.read_csv('/home/vishnu/Desktop/geoserver_links.csv') ## Used sheet is updated in spreadsheet (short descriptions for all layers) at tab(geoserver_links)
print(df)

                   name layer_type             workspace  \
0        admin_boundary     vector  panchayat_boundaries   
1        aquifer_vector     vector               aquifer   
2        drainage_lines     vector              drainage   
3  change_Afforestation     raster      change_detection   

                                layer_name  \
0                       [district]_[block]   
1        aquifer_vector_[district]_[block]   
2                       [district]_[block]   
3  change_[district]_[block]_Afforestation   

                                       geoserver_url  \
0  https://geoserver.core-stack.org:8443/geoserve...   
1  https://geoserver.core-stack.org:8443/geoserve...   
2  https://geoserver.core-stack.org:8443/geoserve...   
3  https://geoserver.core-stack.org:8443/geoserve...   

                                          github_url  
0  https://raw.githubusercontent.com/core-stack-o...  
1  https://raw.githubusercontent.com/core-stack-o...  
2  https://raw.githubu

In [39]:
block_district_state_df = pd.DataFrame({
    'block' : ['gobindpur','mirzapur','koraput','badlapur'],
    'district' : ['saraikela-kharsawan','mirzapur','koraput','jaunpur'],
    'state' : ['jharkhand','uttar_pradesh','odisha','uttar_pradesh']
})

block_district_state_df


,block,district,state
0,gobindpur,saraikela-kharsawan,jharkhand
1,mirzapur,mirzapur,uttar_pradesh
2,koraput,koraput,odisha
3,badlapur,jaunpur,uttar_pradesh


In [62]:
state= 'uttar_pradesh'
district='jaunpur'
block= 'badlapur'
name=['admin_boundary','change_Afforestation','aquifer_vector','drainage_lines']

In [63]:
selected_row = df[df['name'].isin(name)]
print(selected_row)

                   name layer_type             workspace  \
0        admin_boundary     vector  panchayat_boundaries   
1        aquifer_vector     vector               aquifer   
2        drainage_lines     vector              drainage   
3  change_Afforestation     raster      change_detection   

                                layer_name  \
0                       [district]_[block]   
1        aquifer_vector_[district]_[block]   
2                       [district]_[block]   
3  change_[district]_[block]_Afforestation   

                                       geoserver_url  \
0  https://geoserver.core-stack.org:8443/geoserve...   
1  https://geoserver.core-stack.org:8443/geoserve...   
2  https://geoserver.core-stack.org:8443/geoserve...   
3  https://geoserver.core-stack.org:8443/geoserve...   

                                          github_url  
0  https://raw.githubusercontent.com/core-stack-o...  
1  https://raw.githubusercontent.com/core-stack-o...  
2  https://raw.githubu

In [64]:
for index, row in selected_row.iterrows():
    workspace = row['workspace']
    temp_layer_name = row['layer_name']
    geoserver_url = row['geoserver_url']
    
    final_layer_name = temp_layer_name.replace('[district]', district).replace('[block]', block)

    final_url = geoserver_url.replace('[workspace]', workspace).replace('[layer_name]', final_layer_name)
    
    print("Generated URL for",final_layer_name, "---")
    print(final_url) 

Generated URL for jaunpur_badlapur ---
https://geoserver.core-stack.org:8443/geoserver/panchayat_boundaries/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=panchayat_boundaries%3Ajaunpur_badlapur&outputFormat=application%2Fjson
Generated URL for aquifer_vector_jaunpur_badlapur ---
https://geoserver.core-stack.org:8443/geoserver/aquifer/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=aquifer%3Aaquifer_vector_jaunpur_badlapur&outputFormat=application%2Fjson
Generated URL for jaunpur_badlapur ---
https://geoserver.core-stack.org:8443/geoserver/drainage/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=drainage%3Ajaunpur_badlapur&outputFormat=application%2Fjson
Generated URL for change_jaunpur_badlapur_Afforestation ---
https://geoserver.core-stack.org:8443/geoserver/change_detection/wcs?service=WCS&version=2.0.1&request=GetCoverage&CoverageId=change_detection:change_jaunpur_badlapur_Afforestation&format=geotiff
